# Prediction 

In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import geopandas as gpd
import h3
from shapely import wkt
from sklearn.linear_model import Ridge, LinearRegression, RidgeCV, LassoCV, Lasso, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler
import pydeck as pdk
import seaborn as sns
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
df9 = pd.read_csv('model9.csv')
df10 = pd.read_csv('model10.csv')

In [171]:
df10.head()

,date,h6,h7,h10,alerts,stringency,precip,schools,hospitals,stringency^2,stringency^3,precip^2,precip^3,schools^2,schools^3,hospitals^2,hospitals^3,dayofweek,month,HomeGame,IsHoliday,stringency precip,stringency schools,stringency hospitals,stringency stringency^2,stringency stringency^3,stringency precip^2,stringency precip^3,stringency schools^2,stringency schools^3,stringency hospitals^2,stringency hospitals^3,stringency dayofweek,stringency month,stringency HomeGame,stringency IsHoliday,precip schools,precip hospitals,precip stringency^2,precip stringency^3,precip precip^2,precip precip^3,precip schools^2,precip schools^3,precip hospitals^2,precip hospitals^3,precip dayofweek,precip month,precip HomeGame,precip IsHoliday,schools hospitals,schools stringency^2,schools stringency^3,schools precip^2,schools precip^3,schools schools^2,schools schools^3,schools hospitals^2,schools hospitals^3,schools dayofweek,schools month,schools HomeGame,schools IsHoliday,hospitals stringency^2,hospitals stringency^3,hospitals precip^2,hospitals precip^3,hospitals schools^2,hospitals schools^3,hospitals hospitals^2,hospitals hospitals^3,hospitals dayofweek,hospitals month,hospitals HomeGame,hospitals IsHoliday,stringency^2 stringency^3,stringency^2 precip^2,stringency^2 precip^3,stringency^2 schools^2,stringency^2 schools^3,stringency^2 hospitals^2,stringency^2 hospitals^3,stringency^2 dayofweek,stringency^2 month,stringency^2 HomeGame,stringency^2 IsHoliday,stringency^3 precip^2,stringency^3 precip^3,stringency^3 schools^2,stringency^3 schools^3,stringency^3 hospitals^2,stringency^3 hospitals^3,stringency^3 dayofweek,stringency^3 month,stringency^3 HomeGame,stringency^3 IsHoliday,precip^2 precip^3,precip^2 schools^2,precip^2 schools^3,precip^2 hospitals^2,precip^2 hospitals^3,precip^2 dayofweek,precip^2 month,precip^2 HomeGame,precip^2 IsHoliday,precip^3 schools^2,precip^3 schools^3,precip^3 hospitals^2,precip^3 hospitals^3,precip^3 dayofweek,precip^3 month,precip^3 HomeGame,precip^3 IsHoliday,schools^2 schools^3,schools^2 hospitals^2,schools^2 hospitals^3,schools^2 dayofweek,schools^2 month,schools^2 HomeGame,schools^2 IsHoliday,schools^3 hospitals^2,schools^3 hospitals^3,schools^3 dayofweek,schools^3 month,schools^3 HomeGame,schools^3 IsHoliday,hospitals^2 hospitals^3,hospitals^2 dayofweek,hospitals^2 month,hospitals^2 HomeGame,hospitals^2 IsHoliday,hospitals^3 dayofweek,hospitals^3 month,hospitals^3 HomeGame,hospitals^3 IsHoliday,dayofweek month,dayofweek HomeGame,dayofweek IsHoliday,month HomeGame,month IsHoliday,HomeGame IsHoliday,stringency precip schools,stringency precip hospitals,stringency precip stringency^2,stringency precip stringency^3,stringency precip precip^2,stringency precip precip^3,stringency precip schools^2,stringency precip schools^3,stringency precip hospitals^2,stringency precip hospitals^3,stringency precip dayofweek,stringency precip month,stringency precip HomeGame,stringency precip IsHoliday,stringency schools hospitals,stringency schools stringency^2,stringency schools stringency^3,stringency schools precip^2,stringency schools precip^3,stringency schools schools^2,stringency schools schools^3,stringency schools hospitals^2,stringency schools hospitals^3,stringency schools dayofweek,stringency schools month,stringency schools HomeGame,stringency schools IsHoliday,stringency hospitals stringency^2,stringency hospitals stringency^3,stringency hospitals precip^2,stringency hospitals precip^3,stringency hospitals schools^2,stringency hospitals schools^3,stringency hospitals hospitals^2,stringency hospitals hospitals^3,stringency hospitals dayofweek,stringency hospitals month,stringency hospitals HomeGame,stringency hospitals IsHoliday,stringency stringency^2 stringency^3,stringency stringency^2 precip^2,stringency stringency^2 precip^3,stringency stringency^2 schools^2,stringency stringency^2 schools^3,stringency stringency^2 hospitals^2,stringency stringency^2 hospitals^3,stringency stringency^2 dayofweek,stringen

In [6]:
parent_str = 'h6'
child_str = 'h10'
P6_C10 = PC_lasso(df10, df10.h6, parent_str, child_str , numrows_cond = 1000)


In [5]:
class PC_lasso:
    def __init__(self, df, parent, parent_str, child_str ,numrows_cond=0):
        self.parent = parent
        self.data = df
        self.numrows_cond = numrows_cond
        self.hexagons = parent.unique() # number of unique parent hexagons in the data
        
    def fit_each(self):
        model_info = {} 
        for h in self.hexagons:
            hdict = {}
            d=df.loc[self.parent==h, :]
            if d.shape[0]>=self.numrows_cond:
                X = d.iloc[:,5:]
                y = d[['alerts']]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
                lasso = LassoLarsIC(criterion='aic', normalize=True).fit(X_train, y_train)
                hdict['X_test'] = X_test
                hdict['X_train'] = X_train
                hdict['y_train'] = y_train
                hdict['y_test'] = y_test
                hdict['lasso'] = lasso
                hdict['nonzero'] = X.columns[lasso.coef_!=0]
                model_info[h] = hdict
        return model_info

    def display_results(self):
        model_info = self.fit_each()
        for h in model_info:
            print('')
            print('For Hexagon: ' + h)
            print('-------------------------------------------')
            print("R^2: {}".format(lasso.score(X_test, y_test)))
            mse = mean_squared_error(y_test, y_pred)
            print("Mean Squared Error: {}".format(mse))
            print('')
            print('non-zero columns:')
            print(list(model_info[h]['nonzero']))
            print('-------------------------------------------')
            print('')
            print('')
            
            
    def hexagons_used(self):
        hex_list = []
        for h in self.hexagons:
            d=df.loc[self.parent==h, :]
            if d.shape[0]>=1000:
                hex_list.append(h)
        return hex_list
        
    
    def mapall(self):

        hex_list = self.hexagons_used()
        d = self.data[self.data[parent_str].isin(hex_list)]
        d = d[[parent_str, child_str, 'alerts']]
        d = d.reset_index()
        d["count_scaled"] = 4 * MinMaxScaler().fit_transform(d["alerts"].values.reshape(-1, 1))
        layer = pdk.Layer(
        "H3HexagonLayer",
        d,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon=parent_str,
        get_fill_color="[225, (1 - count_scaled) * 255, (1 - count_scaled)/5 * 255]",
        get_line_color="[255, 255, 255]",
        line_width_min_pixels=2.5,
        )
        # Set `ViewState` to center Cluj-Napoca
        view_state = pdk.ViewState(latitude=46.770920, longitude=23.589920, zoom=11, bearing=0, pitch=45)

        # Render with Deck
        r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Count: {count}"})
        return r


## Using h6 as our parent, h10 as child

In [229]:
h6 = df10['h6'].unique()
for i in h6:
    d=df10.loc[df10.h6==i, :]
    if d.shape[0]>=1500:
        X = d.iloc[:,6:]
        y = d[['alerts']]
        print('')
        print('For Hexagon: ' + i)
        print('-------------------------------------------')
        forest = RandomForestRegressor(criterion = 'mse',random_state=0, max_features=1/3)
        print('Cross Validation for scoring')
        scores = cross_val_score(forest,X,y,cv=KFold(5,shuffle=True,random_state=0),scoring="neg_mean_squared_error")
        print('')
        mse = np.mean(scores)
        print('Average OOS mse for 5 folds: ', -mse)
        print('')
        print('')


For Hexagon: 861e0b237ffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.0784490971742353



For Hexagon: 861e0b2a7ffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.2468193098960092



For Hexagon: 861e0b387ffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.8982039544548424



For Hexagon: 861e0b38fffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.905956040769098



For Hexagon: 861e0b397ffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.6324044546727485



For Hexagon: 861e0b39fffffff
-------------------------------------------
Cross Validation for scoring

Average OOS mse for 5 folds:  1.5627542095951896



For Hexagon: 861e0b3afffffff
-----------------------------------------

In [224]:
h6 = df10['h6'].unique()
for i in h6:
    d=df10.loc[df10.h6==i, :]
    if d.shape[0]>=1000:
        print('')
        X = d.iloc[:,6:]
        y = d[['alerts']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
        lasso = LassoLarsIC(criterion='aic', normalize=True).fit(X_train, y_train)
        y_pred = lasso.predict(X_test)
        print('For Hexagon: ' + i)
        print('-------------------------------------------')
        print("R^2: {}".format(lasso.score(X_test, y_test)))
        mse = mean_squared_error(y_test, y_pred)
        print("Mean Squared Error: {}".format(mse))
        print('')
        print('non-zero columns:')
        print(list(X.columns[lasso.coef_!=0]))
        print('-------------------------------------------')
        print('')
        print('')


For Hexagon: 861e0b217ffffff
-------------------------------------------
R^2: -0.023723539205697808
Mean Squared Error: 0.36349390820403404

non-zero columns:
['precip^2 precip^3 HomeGame']
-------------------------------------------



For Hexagon: 861e0b237ffffff
-------------------------------------------
R^2: 0.02067823083002729
Mean Squared Error: 1.1070906714428619

non-zero columns:
['dayofweek', 'IsHoliday', 'stringency stringency^3', 'stringency^2 month', 'dayofweek month', 'dayofweek IsHoliday', 'stringency stringency^3 dayofweek', 'stringency precip^2 dayofweek', 'stringency precip^3 dayofweek', 'precip dayofweek HomeGame', 'stringency^2 dayofweek HomeGame', 'precip^2 precip^3 IsHoliday', 'dayofweek month IsHoliday']
-------------------------------------------



For Hexagon: 861e0b2a7ffffff
-------------------------------------------
R^2: 0.016446388662759492
Mean Squared Error: 1.1411551269259488

non-zero columns:
['stringency^2', 'dayofweek', 'month', 'precip dayofweek'

## Testing

In [9]:
model_info = {} 

test_sets=[]
h6 = df10.h6.unique()
for h in h6:
    hdict = {}
    d=df10.loc[df10.h6==h, :]
    if d.shape[0]>=1000:
        X = d.iloc[:,6:]
        y = d[['alerts']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
        test_sets.append(X_test)
        lasso = LassoLarsIC(criterion='aic', normalize=True).fit(X_train, y_train)
        hdict['lasso'] = lasso
        hdict['OOS predictions'] = lasso.predict(X_test)
        hdict['nonzero'] = X.columns[lasso.coef_!=0]
        model_info[h] = hdict


In [10]:
hex_list = []
for h in h6:
    d=df10.loc[df10.h6==h, :]
    if d.shape[0]>=1000:
        hex_list.append(h)
d = df10[df10['h6'].isin(hex_list)] # dataframe with only hexagons used in the model
d.to_csv()

d = d[['h6', 'h10', 'alerts']]
d = d.reset_index()
d["count_scaled"] = 4 * MinMaxScaler().fit_transform(d["alerts"].values.reshape(-1, 1))
d

,index,h6,h10,alerts,count_scaled
0,0,861e0b217ffffff,8a1e0b210d37fff,1,0.000000
1,1,861e0b217ffffff,8a1e0b21282ffff,1,0.000000
2,2,861e0b217ffffff,8a1e0b214537fff,2,0.111111
3,3,861e0b217ffffff,8a1e0b216b4ffff,1,0.000000
4,4,861e0b217ffffff,8a1e0b38926ffff,1,0.000000
...,...,...,...,...,...
198438,199277,861e0b3afffffff,8a1e0b3ac1a7fff,1,0.000000
198439,199278,861e0b3afffffff,8a1e0b3ac54ffff,2,0.111111
198440,199279,861e0b3afffffff,8a1e0b3ae66ffff,1,0.000000
198441,199281,861e0b3afffffff,8a1e0b3ac187fff,1,0.000000


In [12]:
layer = pdk.Layer(
"H3HexagonLayer",
d,
pickable=True,
stroked=True,
filled=True,
extruded=False,
get_hexagon="h6",
get_fill_color="[225, (1 - count_scaled) * 255, (1 - count_scaled)/5 * 255]",
get_line_color="[255, 255, 255]",
line_width_min_pixels=2.5,
)
# Set `ViewState` to center Cluj-Napoca
view_state = pdk.ViewState(latitude=46.770920, longitude=23.589920, zoom=11, bearing=0, pitch=45)

# Render with Deck
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text":  "Count: {alerts}"})

In [ ]:
r